In [1]:
import pandas as pd
import numpy as np

# 熵权法计算权重

In [2]:
# 读取数据
data = pd.read_excel('../Resource/ExperimentDataBase/Spad_Dta_GRX.xlsx')
data.head()

,SPAD,AGB,LAI,Pn,Ci,Tr,Gs
0,37.466667,1.708889,2.917709,16.573453,263.358997,3.381668,0.178455
1,35.991667,2.063889,2.054150,22.759493,268.242120,5.887681,0.395477
2,35.941667,1.997500,3.599002,16.718943,220.002423,4.542420,0.274608
3,34.941667,1.362222,3.320240,18.022922,196.736997,3.629510,0.170752
4,40.300000,1.965000,3.309667,17.883205,215.082727,4.303128,0.226866


In [3]:
data_n_data = data[['SPAD','AGB','LAI','Ci']]

In [4]:
# 计算决策矩阵X和归一化矩阵Y
X = data_n_data.iloc[:, :].values
Y = np.apply_along_axis(lambda x: x / np.sum(x), 0, X)

In [5]:
# 计算信息熵和权重
m, n = Y.shape
E = (-1 / np.log(m)) * np.sum(Y * np.log(Y), axis=0)
W = (1 - E) / np.sum(1 - E)

In [6]:
# 归一化处理
W = W / np.sum(W)

In [7]:
# 输出结果
for i, w in enumerate(W):
    print(f"第{i+1}列的权重为{w:.3f}")

第1列的权重为0.060
第2列的权重为0.372
第3列的权重为0.315
第4列的权重为0.252
